# Using Kroger's API

Kroger is a midwestern grocery chain. The offer a public API providing information on stores and products, which might be useful for data analisys related to food, prices, and geographic trends.

Documenation is here: https://developer.kroger.com/documentation/public/getting-started/quick-start

You can create a free account to get a username and password to access the API. We only need "Client" credentials. Once you have those, (Copy & Edit) this workbook and store your credentials as "kroger_api_name" and "kroger_api_pw" in [Kaggle User Secrets](https://www.kaggle.com/product-feedback/114053), under the "Add-ons" menu, above.

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
auth = (user_secrets.get_secret("kroger_api_name"),
          user_secrets.get_secret("kroger_api_pw"))

In [ ]:
# Use our stored credentials to get a "token" which authorizes all of our other requests.
# This is easier than regular Oauth because we don't need access to some random user's stuff.
headers = {"Content-Type":"application/x-www-form-urlencoded"}
x = requests.post("https://api.kroger.com/v1/connect/oauth2/token", 
                  data={"grant_type":"client_credentials","scope":"product.compact"}, headers=headers, auth=auth)
token = x.json()['access_token']

In [ ]:
# Here we search for nearby stores based on a zipcode

zipcode_str = "48198"          # We can also seach like: filter.latLong.near=39.306346,-84.278902
search_radius_miles = 20
max_results = 200              # 200 is the maximu allowed
chain = "Kroger"               # The Kroger company owns other grocery chains, gas stations, etc.

headers = {"Authorization": "Bearer " + token}
params = {"filter.zipCode.near":zipcode_str, 
          "filter.limit":max_results,
          "filter.radiusInMiles":search_radius_miles,
          "filter.chain":chain}
resp = requests.get("https://api.kroger.com/v1/locations", headers=headers, params=params)
print("done")

In [ ]:
# The response is JSON. And the "data" part an array of store objects.
stores = resp.json()["data"]
print("Found " + str(len(stores)) + " stores.")

In [ ]:
# Let's look at the first store object
print(json.dumps(stores[0], indent=4))

In [ ]:
# List stores in descending order of how many departments they have

sorted([str(len(store["departments"]))+" departments at store:" +
        store["locationId"]+" "+store["name"]+",   "+store["address"]["addressLine1"]+" "+store["address"]["city"]
        for store in stores],reverse=True)

In [ ]:
# locationId of the Kroger near the main branch of the Ypsi public library. 
whittaker = "01800707"  

In [ ]:
# Convert JSON to a DataFrame
norm = pd.json_normalize(stores)
norm.head()

In [ ]:
# We see above that the departments column contains lists of department objects. 
# Each department object has a "name". And we just want a list of the names.
norm.departments = norm.departments.map(lambda dept_obj_list: [d["name"] for d in dept_obj_list])
norm.head()

In [ ]:
# "explode" the column of lists into separate rows for each element of each list.
departments_df = norm.explode("departments")
# Change the plural column name, since the data is no longer a list.
departments_df.rename(columns={"departments":"department"}, inplace=True)
departments_df

In [ ]:
# Now we can do analysis, like checking which departments are found in all stores and which are less common.

print("Total number of each kind of department across all " + str(len(stores)) + " stores.")
departments_df.department.value_counts()

## Products
Now lets look up some products within stores.

In [ ]:
# Search for "sardines" at a store near me.

headers = {"Authorization": "Bearer " + token}
params = {"filter.locationId": whittaker, "filter.limit":50, "filter.term":"sardines"}
resp = requests.get("https://api.kroger.com/v1/products", headers=headers, params=params)

In [ ]:
# "data" is an array of product objects
products = resp.json()['data']

In [ ]:
# Look at the first product record
products[0]

In [ ]:
# Display the "description" of every product we found.
print("Found "+str(len(products)))
[product['description'] for product in products]

In [ ]:
# How many matches for "sardines" does each nearby store have?

headers = {"Authorization": "Bearer " + token}
for store in stores:
    params = {"filter.locationId": store["locationId"], "filter.limit":50, "filter.term":"sardines"}
    resp = requests.get("https://api.kroger.com/v1/products", headers=headers, params=params)
    products = resp.json()["data"]
    print(store["name"], len(products))
    
print("done.")
    